In [1]:
import pandas as pd
import numpy as np

In [2]:
num = pd.read_csv("numerical(1).csv")
num.shape

(95412, 315)

In [3]:
cat = pd.read_csv("categorical.csv")

In [4]:
# transforming Domain_B 
mapping = {1: 4, 2: 3, 3: 2, 4: 1}
cat["DOMAIN_B"] = cat["DOMAIN_B"].replace(mapping)
cat["DOMAIN_B"].value_counts() 

DOMAIN_B
3    48356
4    28498
2    16754
1     1804
Name: count, dtype: int64

In [5]:
#Converting DATASRCE into object
cat["DATASRCE"] = cat["DATASRCE"].astype('object')

In [6]:
# Dropping RFA_2R because it has only one unique value
cat.drop("RFA_2R",axis =1,inplace=True)

In [7]:
target = pd.read_csv("target.csv")


# Look critically at the dtypes of numerical and categorical columns and make changes where appropriate

In [ ]:
# numer =[]
# for i in num.columns:
#     unique_values = num[i].nunique()
#     if unique_values <=10: 
#         numer.append(i)
# # this above code check the number of unique values of each 
# #numerical columns and when the unique values are less than or equal to 10 , we will convert it into category        

In [ ]:
# for i in numer:
#     num[i] = num[i].astype('object')

In [ ]:
# num.columns[num.dtypes==object]

In [ ]:
# for column in numerical.columns:
#     unique_values = numerical[column].unique()
#     print(f"Column: {column}, Unique Values: {unique_values}")

In [ ]:
# All features in numerical are integer or float.
# Even they are categorical, but have dtype integer or float, I will not convert dtype, as this will create too much additional features for the future model.

# concating num and cat = X

In [8]:
X = pd.concat([num,cat],axis =1)

In [9]:
y =target["TARGET_B"]

In [10]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 1)

In [11]:
#Split further into train_num and train_cat. Also test_num and test_cat
train_cat = X_train.select_dtypes(include = "object")
train_num = X_train.select_dtypes(exclude = "object")
test_cat = X_test.select_dtypes(include = "object")
test_num = X_test.select_dtypes(exclude = "object")

In [12]:
# Scale the features using standardscaler

from sklearn.preprocessing import StandardScaler

transformer = StandardScaler().fit(train_num)
X_train_scale = pd.DataFrame(transformer.transform(train_num),columns = train_num.columns)
X_test_scale = pd.DataFrame(transformer.transform(test_num),columns = test_num.columns)

In [13]:
# Encode the categorical features using One-Hot Encoding

# One hot encoding
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='error', drop='first')
encoder.fit(train_cat)

cols = encoder.get_feature_names_out(input_features=train_cat.columns)

#X_train_one_hot = encoder.transform(train_cat).toarray()
X_train_one_hot = pd.DataFrame(encoder.transform(train_cat).toarray(),columns=cols)
X_test_one_hot= pd.DataFrame(encoder.transform(test_cat).toarray(),columns=cols)




In [14]:
#re-concatenate train_num and train_cat as X_train as well as test_num and test_cat as X_test
X_train_r = pd.concat((X_train_scale,X_train_one_hot),axis=1)
X_test_r = pd.concat((X_test_scale,X_test_one_hot),axis=1)

In [15]:
# Fit a logistic regression (classification) model on the training data.
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [16]:
model = LogisticRegression()
model.fit(X_train_r, y_train)
y_pred = model.predict(X_test_r)

In [17]:
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ",accuracy)


Accuracy:  0.9487501965099827


In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[18105,     0],
       [  978,     0]], dtype=int64)

In [19]:
score = model.score(X_test_r, y_test)
print("Test Accuracy Score", score)


Test Accuracy Score 0.9487501965099827


# Managing imbalance in the dataset

In [20]:
# Check for the imbalance
target['TARGET_B'].value_counts()

TARGET_B
0    90569
1     4843
Name: count, dtype: int64

In [21]:
X_train_r_s = pd.DataFrame(X_train_r)


In [22]:
y_train = y_train.reset_index(drop = True)

In [23]:
data_B = pd.concat([X_train_r_s,y_train],axis =1)
data_B.shape

(76329, 356)

In [24]:
from sklearn.utils import resample

category_0 = data_B[data_B['TARGET_B'] == 0]
category_1 = data_B[data_B['TARGET_B'] == 1]

In [25]:
category_0.shape

(72464, 356)

In [26]:
category_1.shape

(3865, 356)

# Downsampling (undersampling)

In [27]:
category_0_undersampled = resample(category_0,
                                   replace=False,
                                   n_samples = len(category_1))

In [28]:
print(category_0_undersampled.shape)
print(category_1.shape)

(3865, 356)
(3865, 356)


In [29]:
data_downsampled = pd.concat([category_0_undersampled, category_1], axis=0)
data_downsampled['TARGET_B'].value_counts()

TARGET_B
0    3865
1    3865
Name: count, dtype: int64

In [30]:
y_train_d = data_downsampled["TARGET_B"]
data_downsampled.drop("TARGET_B", axis =1,inplace =True)

In [31]:
X_test_r_s = pd.DataFrame(X_test_r)

In [32]:
model1 = LogisticRegression()
model1.fit(data_downsampled, y_train_d)
y_pred1 = model1.predict(X_test_r_s)

C:\Users\Denish\anaconda1\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [33]:
# Calculate accuracy using accuracy_score
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred1)
print("Accuracy:", accuracy)

Accuracy: 0.5695645338783211


In [34]:
# Get the model's score (performance on training data)
score = model1.score(X_test_r_s, y_test)
print("Model score:", score)

Model score: 0.5695645338783211


In [35]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

precision = precision_score(y_test, y_pred1)

recall = recall_score(y_test, y_pred1)

cm = confusion_matrix(y_test, y_pred1)

print("Precision:", precision)
print("Recall:", recall)
print("Confusion matrix:")
print(cm)

Precision: 0.06763862332695984
Recall: 0.5787321063394683
Confusion matrix:
[[10303  7802]
 [  412   566]]


In [36]:
# new_y_pred1 = pd.Series(y_pred1)
# new_y_pred1

# Upsampling (oversampling)

In [37]:
category_1_oversampled = resample(category_1,
                                  replace=True,
                                  n_samples = len(category_0))

In [38]:
print(category_0.shape)
print(category_1_oversampled.shape)

(72464, 356)
(72464, 356)


In [39]:
data_upsampled = pd.concat([category_0, category_1_oversampled], axis=0)
data_upsampled['TARGET_B'].value_counts()

TARGET_B
0    72464
1    72464
Name: count, dtype: int64

In [40]:
data_upsampled

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,TARGET_B
0,-0.054979,-1.360613,0.508956,0.349417,-0.250339,0.188977,-1.087130,1.281545,-1.450164,-0.875158,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
1,-0.055998,-0.003631,0.508956,0.718790,-0.357662,-0.208697,-0.651540,-0.045104,0.243243,-0.420006,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0
2,-0.054979,-0.115145,-1.254835,0.718790,0.930209,-0.208697,-0.825776,1.878537,-1.845292,0.035146,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,-0.054979,-0.003631,-1.254835,-1.866826,-0.357662,-0.208697,-0.303068,1.148880,-0.772801,-1.102734,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0
5,-0.055998,-1.083842,-1.254835,0.718790,-0.357662,-0.208697,0.306758,-0.310433,0.299690,1.400602,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13876,-0.055998,-1.498999,0.508956,0.718790,-0.357662,0.586651,0.045404,-0.111436,0.243243,0.262722,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
69639,-0.053960,-0.668686,1.684816,0.349417,-0.250339,-0.208697,-0.477304,0.286559,-0.942142,1.628178,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
15625,-0.055998,0.023240,-1.842765,0.718790,-0.357662,-0.208697,0.132522,0.087561,0.356137,-0.647582,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1
55231,-0.027471,-1.498999,0.508956,0.718790,-0.357662,-0.208697,0.045404,0.087561,-0.547013,0.262722,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1


In [41]:
y_train_u = data_upsampled["TARGET_B"]
data_upsampled.drop("TARGET_B", axis =1,inplace =True)

In [42]:
model2 = LogisticRegression()
model2.fit(data_upsampled, y_train_u)
y_pred2 = model2.predict(X_test_r)

C:\Users\Denish\anaconda1\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

precision_u = precision_score(y_test, y_pred2)

recall_u = recall_score(y_test, y_pred2)

cm_u = confusion_matrix(y_test, y_pred2)

print("Precision:", precision_u)
print("Recall:", recall_u)
print("Confusion matrix:")
print(cm_u)

Precision: 0.07194050033806626
Recall: 0.5439672801635992
Confusion matrix:
[[11242  6863]
 [  446   532]]


In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             max_samples=0.8, # fraction of X-train to use in each tree
                             random_state=42)
clf.fit(data_upsampled, y_train_u)
print(clf.score(data_upsampled, y_train_u))
print(clf.score(X_test_r, y_test))

y_pred = clf.predict(X_test_r)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.6172306248620004
0.6077660745165855


TARGET_B
0    18105
1      978
Name: count, dtype: int64

array([[11048,  7057],
       [  428,   550]], dtype=int64)

# Feature Selection

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# # scaler = MinMaxScaler()
# # numerical_scaled = scaler.fit_transform(numerical)

# scaler = MinMaxScaler().fit(num)
# numerical_scaled = pd.DataFrame(scaler.fit_transform(num),columns = num.columns)
# #X_test_scale = pd.DataFrame(transformer.transform(test_num),columns = test_num.columns)

In [45]:
#data_upsampled

from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler().fit(data_upsampled)
numerical_scaled = pd.DataFrame(scaler.fit_transform(data_upsampled),columns = data_upsampled.columns)
#X_test_scale = pd.DataFrame(transformer.transform(test_num),columns = test_num.columns)

In [46]:
numerical_scaled.shape

(144928, 355)

In [170]:
X = numerical_scaled
#y = target['TARGET_B']
y = y_train_u

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
kbestdf = SelectKBest(chi2, k=30).fit_transform(X, y)
# Here we choose 10 so that is easier to analyze results later, as we will see
selected = pd.DataFrame(kbestdf)
selected.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.080808,0.136667,0.136000,0.036364,0.060606,0.010101,0.010101,0.111111,0.656566,0.0,...,0.0,0.272727,0.0,0.909091,0.0,1.0,1.0,0.0,0.0,0.0
1,0.272727,0.086333,0.094167,0.145455,0.161616,0.010101,0.020202,0.070707,0.212121,0.0,...,0.0,0.090909,0.5,0.181818,0.0,0.0,1.0,0.0,0.0,0.0
2,0.151515,0.195833,0.197167,0.072727,0.101010,0.000000,0.030303,0.828283,1.000000,0.0,...,0.0,0.545455,0.5,0.181818,0.0,1.0,1.0,0.0,0.0,0.0
3,0.000000,0.068000,0.074833,0.000000,0.000000,0.000000,0.010101,0.030303,0.101010,0.0,...,0.0,0.090909,0.5,0.090909,0.0,1.0,0.0,0.0,1.0,0.0
4,0.000000,0.046500,0.054667,0.000000,0.000000,0.000000,0.010101,0.010101,0.050505,0.0,...,0.0,0.818182,0.0,0.727273,0.0,1.0,0.0,0.0,0.0,0.0


In [171]:
kbest = SelectKBest(chi2, k=30).fit(X, y)
df = pd.DataFrame(data = kbest.scores_, columns = ['score'])
df['Column'] = data_upsampled.columns
df

,score,Column
0,0.839294,TCODE
1,2.213949,AGE
2,33.223460,INCOME
3,5.095618,WEALTH1
4,3.686993,HIT
...,...,...
350,12.234160,GEOCODE2_D
351,33.062241,DOMAIN_A_R
352,54.731847,DOMAIN_A_S
353,0.032299,DOMAIN_A_T


In [172]:
df.sort_values(by = ['score'], ascending = False).head(30)

,score,Column
313,1325.890979,RFA_2F
347,809.822411,RFA_2A_G
345,586.304981,RFA_2A_E
346,360.591493,RFA_2A_F
325,330.320044,LASTDATE_YR
111,188.984070,HVP2
110,188.048842,HVP1
305,182.930724,CARDGIFT
112,167.771978,HVP3
115,138.053224,HVP6


In [180]:
cols = df.sort_values(by = ['score'], ascending = False).head(30)['Column']
col = list(cols)

In [181]:
data_upsampled_fS = data_upsampled[col]

In [182]:
len(data_upsampled_fS.columns)

30

In [183]:
y_train_u.shape

(144928,)

In [184]:
# x_Test_FS =[]
# for i in X_test_r.columns:
#     if i in (data_upsampled_fS.columns):
#         x_Test_FS.append(i)
 
x_Test_FS = []
for i in data_upsampled_fS.columns:
    if i in (X_test_r.columns):
        x_Test_FS.append(i)



In [185]:
len(x_Test_FS)

30

In [186]:
len(data_upsampled_fS.columns)

30

In [187]:
len(data_upsampled_fS)

144928

In [188]:
len(x_Test_FS)

30

In [189]:
new_X_test = X_test_r[x_Test_FS]
new_X_test.shape

(19083, 30)

In [190]:
data_upsampled_fS

,RFA_2F,RFA_2A_G,RFA_2A_E,RFA_2A_F,LASTDATE_YR,HVP2,HVP1,CARDGIFT,HVP3,HVP6,...,NGIFTALL,DOB_MM,DOMAIN_A_S,CARDPROM,NUMPROM,CLUSTER2,STATE_GA,LASTDATE_MM,RP3,ODATEW_MM
0,-0.848148,0.0,1.0,0.0,-0.789611,-0.627068,-0.471415,0.205560,-0.654937,-0.355373,...,0.628067,0.369117,0.0,0.762008,0.783010,-0.779187,0.0,1.021876,0.986094,-0.024062
1,1.016381,0.0,1.0,0.0,1.114181,-0.595678,-0.471415,-1.118228,-0.764218,-0.355373,...,-0.421966,-0.291028,0.0,-0.750375,-0.738902,0.660879,0.0,-0.864236,-1.079913,-0.024062
2,0.084116,0.0,1.0,0.0,1.114181,-0.564288,-0.509039,1.088086,1.284802,-0.355373,...,0.161386,1.359333,0.0,0.412997,0.522111,-1.419216,0.0,-0.864236,1.016476,-0.024062
4,-0.848148,1.0,0.0,0.0,1.114181,-0.627068,-0.509039,-0.897597,-0.873499,-0.355373,...,-1.005317,-0.291028,0.0,-1.448397,-1.434633,0.660879,0.0,-1.100000,-1.626797,-0.024062
5,1.016381,0.0,0.0,0.0,-0.789611,-0.627068,-0.509039,0.205560,-0.928140,-0.355373,...,0.628067,2.349549,0.0,0.412997,-0.304070,1.354243,0.0,0.550348,-1.778709,-0.024062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13876,-0.848148,0.0,0.0,1.0,-0.789611,-0.093438,-0.170422,-0.897597,0.301273,-0.188754,...,-0.888647,-0.621100,1.0,-0.866712,-0.695419,-0.405836,0.0,0.078820,0.682269,-0.024062
69639,0.084116,0.0,1.0,0.0,-0.789611,2.103863,2.689008,1.088086,1.612646,2.310540,...,0.744737,-0.621100,0.0,0.762008,0.739527,-1.419216,0.0,1.021876,1.198771,-0.024062
15625,1.948645,0.0,1.0,0.0,1.114181,-0.627068,-0.509039,-0.235703,-0.846179,-0.355373,...,0.161386,1.689405,0.0,0.296660,0.130762,1.567586,0.0,-1.100000,-1.353355,-0.024062
55231,-0.848148,0.0,0.0,1.0,-0.789611,0.754093,-0.358543,-0.897597,1.476044,-0.355373,...,-1.005317,-0.621100,0.0,-1.564735,-1.478117,-1.419216,0.0,0.786112,0.591122,-0.024062


In [191]:
new_X_test

,RFA_2F,RFA_2A_G,RFA_2A_E,RFA_2A_F,LASTDATE_YR,HVP2,HVP1,CARDGIFT,HVP3,HVP6,...,NGIFTALL,DOB_MM,DOMAIN_A_S,CARDPROM,NUMPROM,CLUSTER2,STATE_GA,LASTDATE_MM,RP3,ODATEW_MM
0,1.016381,0.0,1.0,0.0,-0.789611,-0.658458,-0.509039,-0.456334,-0.928140,-0.355373,...,-0.305296,-0.621100,0.0,-0.285026,-0.478003,-0.779187,0.0,0.550348,0.986094,-0.024062
1,-0.848148,0.0,0.0,1.0,-0.789611,-0.438728,-0.433791,0.205560,1.120881,-0.355373,...,-0.071955,-0.291028,0.0,0.645671,0.391661,-0.192493,0.0,0.078820,0.621505,-0.024062
2,1.016381,1.0,0.0,0.0,1.114181,-0.658458,-0.509039,1.970611,-0.818859,-0.355373,...,1.678099,1.689405,1.0,1.692706,1.565707,-0.619179,0.0,-1.100000,1.107624,-0.024062
3,0.084116,0.0,0.0,1.0,1.114181,-0.627068,-0.509039,-0.456334,-0.846179,-0.355373,...,-0.305296,3.009693,0.0,-0.517700,-0.434520,-0.299165,0.0,-0.864236,-1.201442,-0.024062
4,-0.848148,0.0,1.0,0.0,1.114181,-0.627068,-0.471415,0.646823,-0.463695,-0.355373,...,0.744737,0.039044,0.0,0.762008,0.913459,-0.245829,0.0,-1.335764,1.077241,-0.024062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19078,1.016381,0.0,1.0,0.0,1.114181,-0.595678,-0.471415,-0.676965,-0.381734,-0.355373,...,-0.771977,-0.621100,0.0,-1.099386,-1.130251,0.500871,0.0,-1.335764,-0.593793,-0.024062
19079,-0.848148,0.0,0.0,1.0,-0.789611,-0.658458,-0.509039,-0.456334,-0.928140,-0.355373,...,-0.538636,2.019477,0.0,-0.168689,0.261211,1.514251,0.0,1.021876,-1.748327,-0.024062
19080,-0.848148,0.0,0.0,1.0,1.114181,-0.627068,-0.509039,-0.015071,-0.818859,-0.355373,...,0.044715,-0.621100,0.0,1.343694,0.869976,0.874222,0.0,-1.335764,-0.806471,-0.024062
19081,1.016381,0.0,0.0,0.0,-0.789611,-0.470118,-0.471415,1.749980,-0.108532,-0.355373,...,2.028110,0.699189,0.0,0.063986,0.174245,-0.885858,0.0,0.786112,-0.198822,-0.024062


In [192]:
y_test.shape

(19083,)

# Re-run the Random Forest algorithm to determine if the Feature Selection has improved the results.

In [193]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=9, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             max_samples=0.8, # fraction of X-train to use in each tree
                             random_state=42)
clf.fit(data_upsampled_fS, y_train_u)
print("train score: ",clf.score(data_upsampled_fS, y_train_u))
print("test score: ",clf.score(new_X_test, y_test))

y_pred = clf.predict(new_X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

train score:  0.7142029145506734
test score:  0.6967981973484253


TARGET_B
0    18105
1      978
Name: count, dtype: int64

array([[12825,  5280],
       [  506,   472]], dtype=int64)

In [ ]:
# from the 

In [ ]:
# i can see good improvement after doing feautre selection, our main objective is to reduce False positive 
#after feuture selection FP = 5280 before it was 6870
# we can further reduce FP by doing parameter changes into random forest algorithm and feature slection also
# FP indicated number of people predicted false as a donor but actually they were not donor and we are sending 5280 wrong gift 
#so finally it affect th 